In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
import pandas as pd
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf,split,explode,lit,array
from pyspark.ml.feature import NGram
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType
import numpy as np
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score,f1_score

In [2]:
spark = SparkSession.\
        builder.\
        appName("ml").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

23/01/18 16:01:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_train = spark.read.parquet('hdfs://namenode:9000/ml/train_data')

In [4]:
df_train.createOrReplaceTempView('df_train')

In [5]:
df = df_train.withColumn('comment_term',split(df_train.clean_content, ' ', -1))
def getNGram(n):
    ngram = NGram(n=n)
    ngram.setInputCol("comment_term")
    ngram.setOutputCol("nGrams")
    df_nGram = ngram.transform(df)
    result_nGram = df_nGram.withColumn('word',explode(df_nGram.nGrams))\
        .groupBy(['word'])\
        .count()
    return result_nGram
result_nGram = getNGram(1)

In [6]:
result_nGram_pd = result_nGram.orderBy(col("count").desc()).toPandas()
result_nGram_pd.to_csv(f'sample/train.csv',index=False)

In [7]:
result_nGram.createTempView('result_nGram')

In [8]:
stop_word = spark.sql("""
    select word from result_nGram
    where count < 10
""").toPandas()

In [9]:
stop_word = stop_word['word'].to_list()
dict_stop_word = {x:1 for x in stop_word}

In [10]:
def remove_stop_word(txt):
    txt = txt.strip()
    ls_words = txt.split()
    ls_new_words = []
    for word in ls_words:
        if dict_stop_word.get(word) == None:
            ls_new_words.append(word)
    return ' '.join(ls_new_words)

In [11]:
spark.udf.register("remove_stop_word", remove_stop_word,StringType())

<function __main__.remove_stop_word(txt)>

In [16]:
new_df = spark.sql("""
    select remove_stop_word(clean_content) clean_content,rating,sentiment,label 
    from df_train
""")

In [17]:
df = new_df.withColumn('comment_term',split(new_df.clean_content, ' ', -1))
def getNGram(n):
    ngram = NGram(n=n)
    ngram.setInputCol("comment_term")
    ngram.setOutputCol("nGrams")
    df_nGram = ngram.transform(df)
    result_nGram = df_nGram.withColumn('word',explode(df_nGram.nGrams))\
        .groupBy(['word'])\
        .count()
    return result_nGram
result_nGram = getNGram(1)

In [18]:
result_nGram_pd = result_nGram.orderBy(col("count").desc()).toPandas()
result_nGram_pd.to_csv(f'sample/train1.csv',index=False)

In [20]:
test = new_df.withColumn('comment_term',explode(split(new_df.clean_content, ' ', -1)))
test.createOrReplaceTempView('test')

In [21]:
spark.sql("""
    select count(distinct comment_term) from 
    test
""").show()

+----------------------------+
|count(DISTINCT comment_term)|
+----------------------------+
|                        9922|
+----------------------------+

